In [1]:
import pandas as pd
import numpy as np
import re
import plotly.express as px
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
words = set(nltk.corpus.words.words())

In [2]:
#Import the data and clean the DF
df = pd.read_csv("data/aggregated_DB.csv", low_memory=False)
df = df.drop_duplicates(subset='tweet', keep="first")
df.to_csv("data/aggregated_DB.csv")

df_add = pd.read_csv("data/stored_vars_hr_update_2018.csv",lineterminator='\n')
df = df.append(df_add)
df = df.drop_duplicates(subset='tweet', keep="first")

df = df[['date', 'tweet']]
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')
print(df.shape)

/var/folders/3_/4hp02k9s28l0xngsnvfp3jl40000gn/T/ipykernel_5748/4241433120.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_add)


(1842508, 2)


In [3]:
#___________________________settings for estimation ____________________
#Threshold for sentiment classification => range: (1,0)
continuous_or_discrete = "disc"
threshold = 0.3
#Use Naive Bayes to classify into relevance scores?
bayes = 0 #this option is a work in progress, 1 = YES, 0 = NO
#_______________________________________________________________________

#define a DF to store the last estimation parameters
settings = {'Setting':['cont_or_disc', 'threshold', 'bayes',],
               'value':[continuous_or_discrete, threshold, bayes]}

settings_df = pd.DataFrame(settings)
settings_df.to_csv(f"results/settings_{continuous_or_discrete}.csv")
print(settings_df)

        Setting value
0  cont_or_disc  disc
1     threshold   0.3
2         bayes     0


In [4]:
#Define a function to clean tweet by removing special characters and whitespace using regex
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",str(tweet)) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", str(tweet)) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet)
         if w.lower() in words or not w.isalpha())
    return tweet

df['tweet_clean'] = df['tweet'].apply(cleaner)
df = df.drop('tweet', axis=1)
df = df.drop_duplicates(subset='tweet_clean', keep="first")


In [5]:
#Uncomment the Following two lines if not already downloaded:
#nltk.download('vader_lexicon')
#nltk.download('words')

df = df.reset_index()
list1 = []
for i in df['tweet_clean']:
    list1.append((sid.polarity_scores(str(i)))['compound'])

df['sentiment'] = pd.Series(list1)
df.to_csv('temp/tempsave.csv')

In [5]:
df = pd.read_csv('temp/tempsave.csv')
#Import csv of keywords derived from michigan survey
df_kws = pd.read_csv("kws/kws.csv")
#Create lists of keywords and keyword categories for aggregagation
kw_list = df_kws['Indicator'].values.tolist()
category_list = df_kws['Category'].values.tolist()
category_list = [*set(category_list)]
print(f"The list of aggregate categories is: {category_list}")

#Detect which keyword was used and store in DF
df['Indicator'] = np.nan
for term in kw_list:
    df['Indicator'] = np.where(df['tweet_clean'].str.contains(term, case=False) == True, term, df['Indicator'])

#Merge in df containing keyword categories
df = pd.merge(df_kws, df, on='Indicator')

#After this point you will have a DF with one row for each tweet, each row will have a sentiment score and associated tags
df = df[['date', 'sentiment', 'Indicator', 'Category']]

The list of aggregate categories is: ['personal', 'prices', 'policy', 'general']


In [7]:
#Map continuous score to discrete if the option is selected
if continuous_or_discrete == 'disc':
    df['sentiment_score'] = pd.cut(x=df['sentiment'], 
                                   bins=[-1.1,(-1)*threshold,threshold,1.1],
                                   labels =[-1,0,1])
    #convert the sentiment category to a numerical score in the set  {-1,0,1}
    df['sentiment_score'] = pd.factorize(df['sentiment_score'])[0] - 1

else:
    df['sentiment_score'] = df['sentiment']
    

df.tail()

,date,sentiment,Indicator,Category,sentiment_score
1285540,2023-01-19 23:15:47,-0.6858,Cost of living,prices,-1
1285541,2023-01-19 23:15:27,-0.8779,Cost of living,prices,-1
1285542,2023-01-19 23:15:21,-0.6662,Cost of living,prices,-1
1285543,2023-01-19 23:15:19,0.5781,Cost of living,prices,1
1285544,2023-01-19 23:14:50,0.6994,Cost of living,prices,1


In [8]:
aggregations = ["Category", "Indicator"]

for agg_by in aggregations:
    print(agg_by)
    df_p = df.pivot_table(index='date', columns=f"{agg_by}", values='sentiment_score', aggfunc='mean')
    df_count = df.pivot_table(index='date', columns=f"{agg_by}", values='sentiment_score', aggfunc='count')
    
    df_p.to_csv(f"results/pivot_{agg_by}.csv")
    df_count.to_csv(f"results/pivot_count_{agg_by}.csv")

#The resulting aggregation will be minute by minute counts + scores




Category
Indicator


In [3]:

agg_by = "Category"

type = ["", "_count"]

for type_stat in type:

    df = pd.read_csv(f"results/pivot{type_stat}_{agg_by}.csv")
    print(f"The uncollapsed df is: {df.shape}")
    #Changing the date format such that it can be aggregated using resample 
    df = df.reset_index()
    df_c = df[df['date'].str.contains( "'" )==False ]
    df_c['date'] = pd.to_datetime(df_c['date'], format='%Y-%m-%d %H:%M:%S')
    df_c = df_c.set_index('date')
    
    if type_stat == "_count":
        agg_count = df_c.resample('D').count()
    else:
        agg_means = df_c.resample('D').mean()
        
    print(f"The daily aggregated df is: {agg_means.shape}")



import plotly.io as pio
pio.templates

fig = px.line(agg_means, x=agg_means.index, y=["general"], title=f"Rolling 30 day average of Sentiment {type_stat}",template="simple_white")
fig.update_layout(
    font_family="Georgia",
    font_color="black",
)
fig.show()

fig = px.line(agg_count, x=agg_count.index, y=["general"], title=f"Rolling 30 day average of Sentiment {type_stat}",template="simple_white")
fig.update_layout(
    font_family="Georgia",
    font_color="black",
)
fig.show()


The uncollapsed df is: (1220481, 5)
The daily aggregated df is: (1846, 5)
The uncollapsed df is: (1220481, 5)
The daily aggregated df is: (1846, 5)


In [10]:


# Find the missing dates by comparing the index to a range of all possible dates
all_dates = pd.date_range(agg_means.index.min(), agg_means.index.max())
missing_dates = set(all_dates) - set(agg_means.index)
print(missing_dates)

agg_means.to_csv("temp/test_dates.csv")

set()


In [10]:

#Collect Michigan Consumer Survey data and add onto the dataframe 
collect_mich = 1

if collect_mich:
    df_mich = pd.read_csv("http://www.sca.isr.umich.edu/files/tbmics.csv")
    df_mich['date'] = pd.to_datetime(df['YYYY'].astype(str) + ' ' + df_mich['Month'] + ' 28', format='%Y %B %d')
    df_mich = df_mich.set_index('date')
    df_mich.to_csv('data/mich_sent.csv')
else:
    df_mich = pd.read_csv('data/mich_sent.csv')


df_mich['ICS_ALL'] = 1.5*df_mich['ICS_ALL'] - 250

#Collect both the daily average score and count into one dataframe
merged_df = pd.merge(agg_count, agg_means, left_index=True, right_index=True, how='inner')

#Create a custom score metric and take a moving average
for cat in category_list:   
    merged_df[cat +"_z"] = (merged_df[cat +"_x"]) / (merged_df[cat +"_x"].mean())
    merged_df[cat +"_score"] = merged_df[cat +"_x"]*merged_df[cat +"_y"]
    merged_df[cat +"_score"] = merged_df[cat +"_score"].rolling(30).mean()


#Merge the michigan sentiment with twitter sentiment
merged_plot = pd.merge(merged_df, df_mich, left_index=True, right_index=True, how='left')
#fix missing values due to michigan being monthly and sentiment being daily
merged_plot = merged_plot.interpolate()
merged_plot['total'] = merged_plot['general_score'] + merged_plot['prices_score']

#Plot the two for comparison 
fig = px.line(merged_plot, x=merged_plot.index, y=["general_score","prices_score","total","ICS_ALL"], title=f"Rolling 30 day average of Sentiment {type_stat}",template="simple_white",width=700)
fig.update_layout(
    font_family="Georgia",
    font_color="black",
)
fig.show()


In [12]:
fig = px.line(merged_plot, x=merged_plot.index, y=["total","ICS_ALL"], title=f"Rolling 30 day average of Sentiment {type_stat}",template="simple_white",width=700)
fig.update_layout(
    font_family="Georgia",
    font_color="black",
)
fig.show()

In [12]:
agg_by = "Category"

type = ["", "_count"]

for type_stat in type:

    df = pd.read_csv(f"results/pivot{type_stat}_{agg_by}.csv")
    print(f"The uncollapsed df is: {df.shape}")
    #Changing the date format such that it can be aggregated using resample 
    df = df.reset_index()
    df_c = df[df['date'].str.contains( "'" )==False ]
    df_c['date'] = pd.to_datetime(df_c['date'], format='%Y-%m-%d %H:%M:%S')
    df_c = df_c.set_index('date')
    
    if type_stat == "_count":
        agg = df_c.resample('D').count()
    else:
        agg = df_c.resample('D').mean()
        
    print(f"The daily aggregated df is: {agg.shape}")

    cat_list_roll = []
    for cat in category_list:   
        agg[cat +"_roll"] = agg[cat].rolling(30).mean()
        cat_list_roll.append([cat +"_roll"])

    #print(cat_list_roll)

    agg.to_csv(f"results/current_trend_{continuous_or_discrete}_2.csv")
    fig = px.line(agg, x=agg.index, y=["general_roll","prices_roll","policy_roll"], title=f"Rolling 30 day average of Sentiment {type_stat}")
    fig.show()

#fig = px.line(agg, x=agg.index, y=[df.(*_roll)], title='Category Sentiment')


The uncollapsed df is: (1220481, 5)
The daily aggregated df is: (1846, 5)


The uncollapsed df is: (1220481, 5)
The daily aggregated df is: (1846, 5)


In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

agg_full = agg.apply(lambda row: row.fillna(row.mean()), axis=1)

#prepare the input series (x) by detrending and standardizing
x = StandardScaler().fit_transform(agg_full)

#Calculate the first  principal component & output explained variance
pca = PCA(n_components=1)
df_PC = pca.fit_transform(x)
print(pca.explained_variance_ratio_)
save_date = ['Sentiment']

df_PC_merge = pd.DataFrame(df_PC, columns = ['Sentiment'])

df1 = pd.concat([save_date, df_PC_merge], axis=1)
df_final = df1.set_index('date')
#df_final.to_csv("/tetemp.csv")



[0.5457438]


TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

In [ ]:
import plotly.express as px
df_final.to_csv("export.csv")
df_final['Sentiment'] = df_final['Sentiment']*(-1)
fig = px.line(df_final, x=df_final.index, y=["Sentiment"], title='PC Sentiment')
fig.show()

In [ ]:
#This is used for inspecting the values
start_date = '2017-12-01'
end_date = '2018-01-31'
mask = (df['date'] > start_date) & (df['date'] <= end_date)
df_trim = df.loc[mask]

df_trim = df_trim[df_trim['sentiment']>-0.8]
df_trim = df_trim[['date','tweet_clean','sentiment']].copy()
df_trim.to_csv("inspect_high.csv")

In [ ]:
#This stores obs. per day
df = df.reset_index()
df = df[df['date'].str.contains( "'" )==False ]
df_temp = df[['date', 'counter']]
df_temp['date'] = pd.to_datetime(df_temp['date'], format='%Y-%m-%d %H:%M:%S')

df_temp = df_temp.set_index('date')
df_temp = df_temp.resample('D').sum()
fig2 = px.line(df_temp, x=df_temp.index, y='counter', title='#obs per day')
fig2.show()


In [ ]:
#This removes those that have already been estimated
if re_est != 1:
    df_already_covered = pd.read_csv(f"results/current_trend_{continuous_or_discrete}.csv")
    df_already_covered = df_already_covered.reset_index()
    df_already_covered
    df_trim = pd.DataFrame()
    for i in df.iteritems():
        df_trim = df[~(df['date'].day == df_already_covered['date'](i).day)]

df_trim.head()

